## In this notebook we test the compatibility of the implemented dataset class for GEOM-QM9 and Sphernet

In [8]:
import torch
from torch_geometric.loader import DataLoader
from dig.threedgraph.evaluation import ThreeDEvaluator
from dig.threedgraph.method import run
from dig.threedgraph.method.spherenet import SphereNet
from qm9_dataset import qm9_data_geom

In [2]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda', index=0)

path to the GEOM-QM9 to be passe dto the dataset class

In [3]:
path_to_data = '../../../others_approaches/conformation_generation/GeoMol/data/QM9/'


In [4]:
qm9_set = qm9_data_geom(root= path_to_data)

We test the dataset's structure by accessing a random element

In [5]:
print(qm9_set[2])

Data(x=[13, 44], edge_index=[2, 24], edge_attr=[24, 4], y=[1], pos=[13, 3], z=[13], chiral_tag=[13], name='C#CC#CC#CC#C', boltzmann_weight=[1], degeneracy=[1], mol=<rdkit.Chem.rdchem.Mol object at 0x7fc73e9204f0>, pos_mask=[10])


We prepare for training with SphereNet for a trivial task just to test if the forward path works without errors

In [6]:

split_idx = qm9_set.get_idx_split(len(qm9_set.data.y)-2, train_size=300, valid_size=100, seed=42)

train_dataset, valid_dataset, test_dataset = qm9_set[split_idx['train']], qm9_set[split_idx['valid']], qm9_set[split_idx['test']]


In [9]:
model = SphereNet(energy_and_force=False, cutoff=5.0, num_layers=4, 
        hidden_channels=128, out_channels=1, int_emb_size=64, 
        basis_emb_size_dist=8, basis_emb_size_angle=8, basis_emb_size_torsion=8, out_emb_channels=256, 
        num_spherical=3, num_radial=6, envelope_exponent=5, 
        num_before_skip=1, num_after_skip=2, num_output_layers=3, use_node_features=True)
loss_func = torch.nn.L1Loss()
evaluation = ThreeDEvaluator()

We run the model evaluation and see below that there were no errors in the whole training epoch. we topped the code middway as the main target was to check for errrors in the forward path.

In [10]:
run3d = run()
run3d.run(device, train_dataset, valid_dataset, test_dataset, 
        model, loss_func, evaluation, 
        epochs=2, batch_size=4, vt_batch_size=4, lr=0.0005, lr_decay_factor=0.5, lr_decay_step_size=15)

#Params: 1890118

=====Epoch 1

Training...


/home/aabdel2s/anaconda3/envs/DIG-Stable/lib/python3.8/site-packages/torch_geometric/deprecation.py:13: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
100%|██████████| 75/75 [00:13<00:00,  5.64it/s]



Evaluating...



100%|██████████| 25/25 [00:01<00:00, 17.45it/s]



Testing...



  0%|          | 166/130431 [00:09<2:01:52, 17.81it/s]


KeyboardInterrupt: 